# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [1]:
# replace the code below with your pulse rate algorithm.
import glob

import numpy as np
import scipy as sp
import scipy.io
import matplotlib.pyplot as plt

# Sampling frequency said
fs = 125


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl):
    '''Takes the data and the reference one and compares the values to measure the error and the confidence.
    Both are returned.'''
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    bpm = sp.io.loadmat(ref_fl)['BPM0']
    
    # Compute pulse rate estimates and estimation confidence.
    ppg = bandpass_filter(ppg)
    accx = bandpass_filter(accx)
    accy = bandpass_filter(accy)
    accz = bandpass_filter(accz)
    
    # Take the frequency of the accelerometer by combining the three signals
    acc = (accx**2 + accy**2 + accz**2)**(1/2)
    
    window_shift = 2*fs
    window_length = 8*fs
    
    bpms = list()
    confidence_list = list()
    errors_list = list()
    # We start with a previous of 100 which is a common value
    previous = 100
    for i in range(0, len(ppg)-window_length, window_shift):
        part_ppg = ppg[i:i+window_length]
        part_acc = acc[i:i+window_length]
    
        freq, freqs, fft = heart_rate_calc(ppg=part_ppg, acc=part_acc, fs=fs, prev = previous)
        previous = freq
        confidence_list.append(confidence_calc(freqs, fft, freq))
        bpms.append(freq)
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    confidence = np.array(confidence_list)
    errors = np.array(np.abs(np.diag(np.subtract(bpms, bpm))))
    return errors, confidence

import scipy.signal

def bandpass_filter(signal, band_pass = (70/60, 180/60), fs = fs):
    """ Performs a bandpass filter on the signal. """
    
    b,a = scipy.signal.butter(3, band_pass, fs=fs, btype= 'bandpass')
    
    # Perform forward and backward digital butterworth filter
    return scipy.signal.filtfilt(b,a,signal)

def fourier_transform(signal, fs):
    '''Calculates the fourier transform for the signal (frequencies + fft)'''
    freqs = np.fft.rfftfreq(len(signal), 1/fs)
    fft = np.abs(np.fft.rfft(signal))
    return freqs, fft

def heart_rate_calc(ppg, acc, fs, prev):
    
    """Takes the ppg, acc, fs and the previous frequency and calculates the heart rate depending on the
    Fourier Transform and the previous frequency. It returns the frequency measured and the fft and frequencies of FFT."""
    
    # Take the peaks of the fft
    freq_ppg, fft_ppg = fourier_transform(ppg, fs)
    
    # Delete the fft at frequencies out of the range 0.4Hz to 4Hz
    fft_ppg[freq_ppg <= 70/60.0] = 0.0
    fft_ppg[freq_ppg >= 180/60.0] = 0.0
    
    # Calculate the peaks
    peaks_ppg = sp.signal.find_peaks(fft_ppg)[0]
    freq_peaks_ppg = freq_ppg[peaks_ppg]
    
    # Same with accelerometer signal
    freq_acc, fft_acc = fourier_transform(acc, fs)
    
    # Delete the fft at frequencies out of the range 0.4Hz to 4Hz
    fft_acc[freq_acc <= 70/60.0] = 0.0
    fft_acc[freq_acc >= 180/60.0] = 0.0
    
    # Calculate the peaks
    peaks_acc = sp.signal.find_peaks(fft_acc)[0]
    freq_peaks_acc = freq_acc[peaks_acc]
    
    # The limit of frequency from the previous point is 11 bpm
    limit = 11/60
    delta = 2/60
    
    #Create a dictionary with frequencies and each magnitude and sort aftwerwards for ppg and accelerometer
    ppg_dict = dict(zip(freq_ppg, fft_ppg))
    sort_ppg_dict = dict(sorted(ppg_dict.items(), key=lambda item: item[1], reverse=True))
    
    acc_dict = dict(zip(freq_ppg, fft_ppg))
    sort_acc_dict = dict(sorted(acc_dict.items(), key=lambda item: item[1], reverse=True))
    
    # Take the frequency without bpm
    prev = prev/60
    
    # Calculates the frequency depending on the limit set between ppg and accelerometer
    #if np.abs(list(sort_ppg_dict.keys())[0] - list(sort_acc_dict.keys())[0]) > limit:
    if np.abs(list(sort_ppg_dict.keys())[0] - prev) > limit:
        dif = list(sort_ppg_dict.keys())[0] - prev
        if dif>0:
            sel_freq = prev+delta
        else:
            sel_freq = prev-delta
    else:
        sel_freq = list(sort_ppg_dict.keys())[0]
    
    # If they are similar both frequencies, it takes the six biggest and measures the frequency
    #else:
        
        #keys_sort_list = list(sort_ppg_dict.keys())
        #print([i*60 for i in [np.abs(prev-keys_sort_list[i]) for i in range(1,8)]])
        #argument = np.argmin(np.array([np.abs(prev-keys_sort_list[i]) for i in range(1,6)]))
        #sel_freq = list(sort_ppg_dict.keys())[argument+1]

    sel_freq = sel_freq*60
    
    return sel_freq, freq_ppg, fft_ppg

def confidence_calc(freqs, fft_mag, bpm_f):
    '''Function that calculates the cofidence interval taking those ones which are 15 bpm '''
    freqs = freqs*fs
    window_f = 15/60
    fundamental_freq_window = (freqs > bpm_f - window_f) & (freqs < bpm_f + window_f)
    return np.sum(fft_mag[fundamental_freq_window])/ np.sum(fft_mag)